## LIBRERIAS NECESARIAS PARA EL PROGRAMA 

In [1]:
# Se cargan las librerias 
import numpy as np
import networkx as nx
import EoN
import matplotlib.pyplot as plt
import pandas as pd 
from   networkx.algorithms import community
import matplotlib.cm as cm
import random

## FORMATO DE LOS DATOS QUE SE USA PARA CREAR EL GRAFO 

In [2]:
## Se cargan los datos de las aristas
aristas = pd.read_excel('red_paula_basica.xlsx')

## CLASES Y FUNCIONES PARA LA IMPLEMENTACION DEL ALGORITMO 

In [3]:
# ---------------------- CLASE PARA LA SIMULACION Y CALCULO DE LA PROBABILIDAD DE CONTAGIO -------------------------------
class simulacion_sir():
    def __init__(self, G, tau, gamma, infected):
        
    # DATOS DE ENTRADA
    # G        : Grafo                (grafo de networkx) 
    # tau      : tasa de contagio     (flotante)
    # gamma    : tasa de recuperacion (flotante)
    # infected : infectados iniciales (lista, iterable)
    # REFERENCIAS
    #  1) Infection in Social Networks: Using Network Analysis to Identify High-Risk
    #     Individuals. R. M. Christley, G. L. Pinchbeck, R. G. Bowers, D. Clancy, N. P. French, R. Bennett, and
    #     Turner. American Journal of Epidemiology. DOI: 10.1093/aje/kwi308
    
    # Parametros de la clase
        N             = len(G.nodes())      # Poblacion total 
        n             = len(infected)       # Poblacion inicial infectada
        self.n        = n                   # Infectados iniciales
        self.tau      = tau                 # Tasa de transmision  
        self.gamma    = gamma               # Tasa de recuperacion 
        self.R0       = tau/gamma           # Promedio de contagios por persona
        self.G        = G                   # Red (grafo)
        self.infected = infected            # Infectados iniciales
        
    # funcion que calcula retorna los nodos infectados en una simulacion
    def infected_nodes(self):
        
        # DATOS DE SALIDA : finalI (diccionario con los nodos infectados)
        
        finalI = EoN.get_infected_nodes(self.G, self.tau, self.gamma, initial_infecteds = self.infected)
        return(finalI)
    
    # funcion que calcula la probabilidad de contagio
    def probabildad_contagio(self, nodo, NpersonasI = 1, Nsim = 1000):
        
        # DATOS DE ENTRADA: 
        # nodo : nodo al que se le calcula la probabilidad de contagio (lista con el nombre del nodo)
        # Nsim : numero de simulaciones (Entero)
        # NpersonasI : Cantidad inicial de personas infectadas (Entero)
        # DATOS DE SALIDA : probabilidad_contagio (flotante)
        
        nodo       = nodo[0]                # Nodo que se desea conocer
        cont       = 0                      # Contador 
        Nodospos   = list(self.G.nodes)     # Nodos que pueden realizar el contagio
        Nodospos.pop(Nodospos.index(nodo))  
        
        for i in range(Nsim):
            #simulacion = simulacion_sir(self.G, self.tau, self.gamma, random.sample(Nodospos,NpersonasI))
            self.infected = random.sample(Nodospos,NpersonasI)
            nodosI        = self.infected_nodes()
            if (nodo in nodosI):
                cont += 1
        probabilidad_contagio = cont/Nsim
        return(probabilidad_contagio)
    
    # funcion para graficar la red 
    def dibujar_red(self):
        nx.draw(self.g, with_labels = True)
        

# ---------------------- FUNCION PARA LA CREACION DEL GRAFO -------------------------------------------------------
def crear_grafo(aristas): 
    # DATOS DE ENTRADA: Data frame de pandas con las aristas y los pesos de la red 
    
    aristas_array = aristas[['Origen','Destino','Peso']].to_numpy()
    G             = nx.Graph()
    for i in range(len(aristas_array)):
        G.add_edge(aristas_array[i][0], aristas_array[i][1], weigth= aristas_array[i][2])
    return(G)

## APLICACION DEL METODO 

In [4]:
# Se crea al grafo y la clase simulacion 
G          = crear_grafo(aristas) 
simulacion = simulacion_sir(G,tau = 1.2, gamma = 1, infected = random.sample(G.nodes,1))

In [5]:
# Estimacion de la probabilidad de contagio 
nodo_principal = [3005105534]
p_contagio     = simulacion.probabildad_contagio(nodo_principal, NpersonasI = 1, Nsim = 1000)
print('La probabilidad de contagio es',p_contagio)

La probabilidad de contagio es 0.607
